In [13]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.table import Table
import uuid
import seaborn as sns
from math import pi
import os


In [16]:
# Chemin du dossier où tu veux stocker les images
output_folder = "images1"


# Créer le dossier s'il n'existe pas
os.makedirs(output_folder, exist_ok=True)


In [17]:
data = pd.read_json("results.json")
data

,a1,a2,a3,b1,b2,c1,d1,e1,f1
description,"Self-knowledge Recall,",Dialogue Recall,Reflection Recall,Context Queries Relevancy,Responder Queries Relevancy,Context Accuracy,Reflection Relevancy,Message Relevancy,Plan Relevancy
archetypes,"{'nerd': {'total_score': 7, 'total_detailed_sc...","{'nerd': {'total_score': 12, 'total_detailed_s...","{'nerd': {'total_score': 13, 'total_detailed_s...",{'nerd': {'std': {'baseline': 0.01428276579827...,{'nerd': {'std': {'baseline': 0.01304950937628...,{'nerd': {'shared_keyword_ratio': 0.1274038461...,"{'nerd': {'total_scores': {'personality': 25, ...","{'nerd': {'total_scores': {'personality': 29, ...",{'debunker': {'total_scores': {'personality': ...


In [18]:
# Archetypes
archetypes = ["nerd", "debunker", "peacekeeper", "chameleon", "troll"]

In [20]:

# A1, A2, A3: 
for task in ["a1", "a2", "a3"]:
    total_scores = [data[task]["archetypes"][arch]["total_score"] for arch in archetypes]
    detailed_scores = [data[task]["archetypes"][arch]["total_detailed_score"] for arch in archetypes]
    max_score = data[task]["archetypes"][archetypes[0]]["max_score"]

    fig, ax = plt.subplots(figsize=(10, 6))
    x = np.arange(len(archetypes))
    width = 0.35

    ax.bar(x - width/2, total_scores, width, label="Total Score", color="skyblue")
    ax.bar(x + width/2, detailed_scores, width, label="Total Detailed Score", color="lightcoral")
    ax.axhline(y=max_score, color="gray", linestyle="--", label=f"Max Score ({max_score})")

    ax.set_xlabel("Archetypes")
    ax.set_ylabel("Scores")
    ax.set_title(f"{data[task]['description']} Scores by Archetype")
    ax.set_xticks(x)
    ax.set_xticklabels(archetypes, rotation=45)
    ax.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(output_folder,f"{task}_scores.png"))
    plt.close()



In [ ]:
# C1: Combiner
keyword_ratios = [data["c1"]["archetypes"][arch]["shared_keyword_ratio"] for arch in archetypes]
cosine_baseline = [data["c1"]["archetypes"][arch]["cosine_similarity"]["baseline"] for arch in archetypes]
cosine_agent = [data["c1"]["archetypes"][arch]["cosine_similarity"]["agent"] for arch in archetypes]

fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(archetypes))
width = 0.25
ax.bar(x - width, keyword_ratios, width, label="Shared Keyword Ratio", color="lightgreen")
ax.bar(x, cosine_baseline, width, label="Cosine Similarity (Baseline)", color="skyblue")
ax.bar(x + width, cosine_agent, width, label="Cosine Similarity (Agent)", color="lightcoral")
ax.set_xlabel("Archetypes")
ax.set_ylabel("Value")
ax.set_title("Context Accuracy: Metrics by Archetype")
ax.set_xticks(x)
ax.set_xticklabels(archetypes, rotation=45)
ax.legend()
plt.tight_layout()
plt.savefig((os.path.join(output_folder,"c1_combined.png")))
plt.close()

In [ ]:
# D1: Grouper histogram avec archetypes par colors
metrics = ["personality", "dialogue"]
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]  # Colors for nerd, debunker, peacekeeper, chameleon, troll

fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(metrics))
width = 0.15  # Width of each bar

# Plot bars for each archetype
for i, arch in enumerate(archetypes):
    scores = [data["d1"]["archetypes"][arch]["average_scores"][metric] for metric in metrics]
    ax.bar(x + i * width, scores, width, label=arch.capitalize(), color=colors[i])

ax.set_xlabel("Evaluation Axes")
ax.set_ylabel("Average Score")
ax.set_title("Reflection Relevancy: Scores by Archetype")
ax.set_xticks(x + width * (len(archetypes) - 1) / 2)
ax.set_xticklabels(metrics)
ax.set_ylim(0, 3.5)  # Set y-axis limit based on data range (0 to 3, with some padding)
ax.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_folder,"d1_grouped_histogram.png"))
plt.close()

In [19]:

# E1, F1: Heatmaps
for task in ["e1", "f1"]:
    axes = list(data[task]["archetypes"][archetypes[0]]["average_scores"].keys())
    heatmap_data = np.array([[data[task]["archetypes"][arch]["average_scores"][axis] for axis in axes] for arch in archetypes])
    
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(heatmap_data, annot=True, fmt=".2f", cmap="YlGnBu", xticklabels=axes, yticklabels=archetypes)
    ax.set_title(f"{data[task]['description']}: Average Scores by Archetype and Axis")
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder,f"{task}_heatmap.png"))
    plt.close()
